# TT-NN

### Introduction
You don’t need to be proficient in C++ or think about circular buffers to program Tenstorrent devices, just like you don’t need to know CUDA to use PyTorch, or C to use numpy!

Meet ttnn - a library for tensor manipulation.

### Source

Like the rest of the Tenstorrent stack, it is open source!

You can find the code [here](https://github.com/tenstorrent/tt-metal/tree/main/ttnn)

### Docs
You can also find more documentation and API reference [here](https://docs.tenstorrent.com/tt-metal/latest/ttnn/index.html)

## Usage

TTNN's internals are implemented in C++, but the main way to interact with it is from Python

The library is already installed in your environments!

In [ ]:
import ttnn

Start by opening the Tenstorrent device you will be using

In [ ]:
device = ttnn.open_device(device_id=0)

### Tensor creation and movement

You can create a tensor on host

In [ ]:
host_tensor = ttnn.full([10, 15], 1.0)
host_tensor

then move it to device

In [ ]:
device_tensor = ttnn.to_device(host_tensor, device)
device_tensor

or directly create the tensor on device

In [ ]:
device_tensor_2 = ttnn.rand([10, 15], device=device)
device_tensor_2

You can also create a ttnn tensor from a torch tensor!

In [ ]:
import torch


In [ ]:
torch_tensor = torch.rand([10, 15])

In [ ]:
host_ttnn_from_torch = ttnn.from_torch(torch_tensor)
host_ttnn_from_torch

In [ ]:
device_ttnn_from_torch = ttnn.from_torch(torch_tensor, device=device, layout=ttnn.TILE_LAYOUT)
device_ttnn_from_torch

Sending tensors back is also just as easy

In [ ]:
device_tensor = ttnn.rand([10, 15], device=device)

In [ ]:
host_tensor = ttnn.from_device(device_tensor)
# Alternative - device_tensor.cpu()
host_tensor

And moving tensors back to torch

In [ ]:
torch_tensor = ttnn.to_torch(device_tensor)
torch_tensor

In [ ]:
print(host_tensor.shape)
print(host_tensor.layout)
print(host_tensor.dtype)
print(host_tensor.memory_config())
print(host_tensor.device())
print(device_tensor.device())

## Tensor layout

[Docs about layouts](https://docs.tenstorrent.com/tt-metal/latest/ttnn/ttnn/tensor.html#layout)

ttnn.ROW_MAJOR_LAYOUT

![](./media/tensor_with_row_major_layout.png)

ttnn.TILE_LAYOUT

![](./media/tensor_with_tile_layout.png)

As you may remember from earlier sections, Tenstorrent devices operate efficiently on tiled data.

Many operations require the inputs to be tilized, not row-major. 

You can change the layout, or choose it when creating or moving the tensor.

The tensor gets padded to fill the tiles, but this is transparent to the user.

Tensors are usually created in row-major layout

In [ ]:
print(ttnn.full([3,4], 1.0).layout)
print(ttnn.full([3,4], 1.0, device=device).layout)

except in some cases

In [ ]:
ttnn.rand([10,15], device=device).layout

and maintain their layout when moved to device

In [ ]:
host_tensor = ttnn.full([3, 4], 1.0)
print(host_tensor.layout)
device_tensor = ttnn.to_device(host_tensor, device)
print(device_tensor.layout)

unless explicitly converted

In [ ]:
print(device_tensor.layout)
device_tensor = ttnn.to_layout(device_tensor, ttnn.TILE_LAYOUT)
print(device_tensor.layout)

Torch tensors are row-major, but you can tilize during the conversion

In [ ]:
torch_tensor = torch.rand([10,15])
print(ttnn.from_torch(torch_tensor).layout)
print(ttnn.from_torch(torch_tensor, device=device).layout)
print(ttnn.from_torch(torch_tensor, device=device, layout=ttnn.TILE_LAYOUT).layout)

## Data types

Supported data types:
- uint16
- uint32
- float32
- bfloat16
- bfloat8_b
- bfloat4_b



In [ ]:
x_bf16 = ttnn.rand([1000, 1000], device=device, dtype=ttnn.bfloat16)
print(x_bf16)

x_float32 = ttnn.typecast(x_bf16, ttnn.float32)
print(x_float32)

x_uint16 = ttnn.typecast(x_bf16, ttnn.uint16)
print(x_uint16)

x_bf8_b = ttnn.typecast(x_bf16, ttnn.bfloat8_b)
print(x_bf8_b)

x_bf4_b = ttnn.typecast(x_bf16, ttnn.bfloat4_b)
print(x_bf4_b)

![](./media/bf16.png)
![](./media/bf8.png)

### Tensor operations

_Note_:

Most operations are only supported on device, not on host.



To find out more about controlling operation math fidelity and limitations, such as TF32-like matrix multiplication of FP32 inputs, see [details](https://github.com/tenstorrent/tt-metal/blob/main/tech_reports/matrix_engine/matrix_engine.md)

Many operations you know and love from pytorch are already here!

In [ ]:
x = ttnn.arange(start=0, end=100, device=device, layout=ttnn.TILE_LAYOUT)
x = ttnn.divide(x, 100)
x = x.reshape([1, 100])

In [ ]:
y = ttnn.rand([1, 100], device=device)
y

In [ ]:
x + y
x * y
x - y
ttnn.divide(x, y)

In [ ]:
ttnn.sin(x)
ttnn.cos(x)
ttnn.exp(x)
ttnn.log(x)
ttnn.sqrt(x)
ttnn.pow(x, 2)

In [ ]:
ttnn.sort(y)

In [ ]:
ttnn.concat([x, y], dim=1)

Slicing also works!

In [ ]:
x[:, 50:100]

And many, many more!

You can find the full set of supported operations [here](https://docs.tenstorrent.com/tt-metal/latest/ttnn/ttnn/api.html#operations)

### TTNN neural network operations

TTNN provides neural network operations as pure functions, similar to `torch.nn.functional`. This lets you structure your neural network module classes however you like!

In [ ]:
input_ids = ttnn.from_torch(
    torch.randint(0, 1000, (2, 32)), dtype=ttnn.uint32, device=device
)
emb_weight = ttnn.rand((1, 1, 1000, 512), dtype=ttnn.bfloat16, device=device)

x = ttnn.embedding(input_ids, emb_weight, layout=ttnn.TILE_LAYOUT)  # [2, 32, 512]
x = ttnn.reshape(x, (2, 1, 32, 512))

# LayerNorm
x = ttnn.layer_norm(x, epsilon=1e-5)

# Linear: 512 -> 2048 -> 512
w1 = ttnn.rand(
    (1, 1, 512, 2048), dtype=ttnn.bfloat16, layout=ttnn.TILE_LAYOUT, device=device
)
x = ttnn.relu(ttnn.linear(x, w1))
w2 = ttnn.rand(
    (1, 1, 2048, 512), dtype=ttnn.bfloat16, layout=ttnn.TILE_LAYOUT, device=device
)
x = ttnn.linear(x, w2)

Let's explore some commonly used neural network operations in more detail:

For a complete list of all available operations, see the [TTNN API documentation](https://docs.tenstorrent.com/tt-metal/latest/ttnn/ttnn/api.html).


## Compilation and compilation cache

You may notice that the operations seem slow when you first run them, and fast the following times

In [ ]:
import time
x = ttnn.rand([1000, 1000], device=device)
start = time.time()
y = ttnn.softmax(x, dim=1)
# because we're not reading back the result,
# we need to synchronize to actually measure the execution time,
# not just the time taken to dispatch the operation
ttnn.synchronize_device(device)
end = time.time()
print(f"First iteration: {end - start} seconds")
start = time.time()
y = ttnn.softmax(x, dim=1)
ttnn.synchronize_device(device)
end = time.time()
print(f"Time taken: {end - start} seconds")


This is because when you first run an operation for a given tensor shape, the underlying tt-metal kernel gets compiled. The following runs re-use the same binary.

If a compile-time argument changes, such as tensor shape, a new compilation is needed

In [ ]:
# Same operation, different shape
x = ttnn.rand([1337, 1337], device=device)
start = time.time()
y = ttnn.softmax(x, dim=1)
ttnn.synchronize_device(device)
end = time.time()
print(f"First iteration: {end - start} seconds")
start = time.time()
y = ttnn.softmax(x, dim=1)
ttnn.synchronize_device(device)
end = time.time()
print(f"Time taken: {end - start} seconds")

## Direct SRAM (L1) control

As explained in previous sections, with tt-metal and tt-nn the user has full control over moving the data into and out of faster, but limited SRAM memory, also known as L1.

Size of SRAM:

Wormhole (n150): 108MB

Wormhole (n300): 192MB

Blackhole (p100a): 180MB

Blackhole (p150a): 210MB

In [ ]:
dram_tensor = ttnn.rand([4096, 4096], device=device)
dram_tensor.memory_config()

In [ ]:
sram_tensor = ttnn.to_memory_config(dram_tensor, ttnn.L1_MEMORY_CONFIG)
sram_tensor.memory_config()

In [ ]:
# warmup, compilation
ttnn.sum(dram_tensor, dim=0)
ttnn.sum(sram_tensor, dim=0)
ttnn.synchronize_device(device)
start = time.time()
for _ in range(10):
    ttnn.sum(dram_tensor, dim=0)
ttnn.synchronize_device(device)
end = time.time()
print(f"DRAM Time taken: {end - start} seconds")
start = time.time()
for _ in range(10):
    ttnn.sum(sram_tensor, dim=0)
ttnn.synchronize_device(device)
end = time.time()
print(f"SRAM Time taken: {end - start} seconds")

When doing a series of operations, deallocate tensors manually to free up memory. This is especially important for the limited L1.

In [ ]:
ttnn.deallocate(sram_tensor)

For even better performance, you can shard the L1 tensor to keep the data closer to the cores processing it - learn more [here](https://docs.tenstorrent.com/tt-metal/latest/ttnn/ttnn/tensor.html#tensor-sharding) and [here](https://github.com/tenstorrent/tt-metal/blob/main/tech_reports/tensor_sharding/tensor_sharding.md)

In [ ]:
from IPython.display import Image, display

width = 500
print("NoC:")
display(Image(filename='./media/noc.png', width=width))

print("Interleaved tensor:")
display(Image(filename='./media/interleaved_rm.png', width=width))

print("Width sharded tensor:")
display(Image(filename='./media/width_sharding.png', width=width))

print("Height sharded tensor:")
display(Image(filename='./media/height_sharding.png', width=width))

print("Block sharded tensor:")
display(Image(filename='./media/block_sharding.png', width=width))

print("ND Sharded tensor:")
display(Image(filename='./media/nd_sharding_0.png', width=width))
display(Image(filename='./media/nd_sharding_1.png', width=width))



In [ ]:
sharded_tensor = ttnn.to_memory_config(dram_tensor, ttnn.L1_MEMORY_CONFIG)
ttnn.sum(sharded_tensor, dim=0)
ttnn.synchronize_device(device)

start = time.time()
for _ in range(10):
    res = ttnn.sum(sharded_tensor, dim=0)
ttnn.synchronize_device(device)
end = time.time()

interleaved_l1_time = end - start
print(f"Interleaved L1 Time taken: {interleaved_l1_time * 1000} ms")
ttnn.deallocate(sharded_tensor)

sharded_config = ttnn.create_sharded_memory_config(
    shape=dram_tensor.shape,
    core_grid=ttnn.CoreGrid(x=8, y=8),
    strategy=ttnn.ShardStrategy.WIDTH,
)

sharded_tensor = ttnn.to_memory_config(dram_tensor, sharded_config)
ttnn.sum(sharded_tensor, dim=0)
ttnn.synchronize_device(device)

start = time.time()
for _ in range(10):
    res = ttnn.sum(sharded_tensor, dim=0)
ttnn.synchronize_device(device)
end = time.time()

width_sharded_time = end - start
print(f"Width sharded Time taken: {width_sharded_time * 1000} ms")
ttnn.deallocate(sharded_tensor)

sharded_config = ttnn.create_sharded_memory_config(
    shape=dram_tensor.shape,
    core_grid=ttnn.CoreGrid(x=8, y=8),
    strategy=ttnn.ShardStrategy.HEIGHT,
)
sharded_tensor = ttnn.to_memory_config(dram_tensor, sharded_config)
ttnn.sum(sharded_tensor, dim=0)
ttnn.synchronize_device(device)

start = time.time()
for _ in range(10):
    res = ttnn.sum(sharded_tensor, dim=0)
ttnn.synchronize_device(device)
end = time.time()

height_sharded_time = end - start
print(f"Height sharded Time taken: {height_sharded_time * 1000} ms")
ttnn.deallocate(sharded_tensor)

Manual control over L1 lets you keep intermediate results in the cache without fusing operations

In [ ]:
x = ttnn.rand([32, 128], device=device, memory_config=ttnn.L1_MEMORY_CONFIG)

In [ ]:
w1 = ttnn.rand([128, 128], device=device, memory_config=ttnn.L1_MEMORY_CONFIG)
w2 = ttnn.rand([128, 128], device=device, memory_config=ttnn.L1_MEMORY_CONFIG)

In [ ]:
x1 = ttnn.linear(x, w1, memory_config=ttnn.L1_MEMORY_CONFIG)
print(x1.memory_config())
x2 = ttnn.relu(x1) # automatically maintains L1 config
print(x2.memory_config())
x3 = ttnn.linear(x2, w2, memory_config=ttnn.L1_MEMORY_CONFIG)
print(x3.memory_config())

ttnn.deallocate(x1)
ttnn.deallocate(x2)
ttnn.deallocate(x3)


## Inference only

You may notice we did not mention autograd - TT-NN is focused on inference.

Support for training is being developed in a separate framework - have you seen our talk?

## Tooling

TT-NN also includes a lot of convenient dev tools, like
- [ttnn-visualizer](https://docs.tenstorrent.com/tt-metal/latest/ttnn/ttnn/tutorials/2025_dx_rework/ttnn_visualizer.html)
- Host and Device [profiling with tracy](https://docs.tenstorrent.com/tt-metal/latest/ttnn/ttnn/profiling_ttnn_operations.html)
- [TT-NN Graph Trace](https://github.com/tenstorrent/tt-metal/blob/main/tech_reports/ttnn/graph-tracing.md)

# Extra

## Task: implement composite SDPA

In [ ]:
import math

# SDPA(Q, K, V) = softmax((Q × K^T) / √d_k) × V
def composite_sdpa(q, k, v, causal_mask, scale=None):
    # TODO: implement this :)
    q_scaled = ...
    k_t = ttnn.permute(k, (0, 1, 3, 2))
    attn_scores = ...
    output = ...
    return output

In [ ]:
import time

batch, num_heads, seq_len, head_dim = 1, 32, 1024, 128
num_iterations, warmup_iterations = 50, 1

print(f"Config: B={batch}, H={num_heads}, S={seq_len}, D={head_dim}, Causal=True")

torch.manual_seed(42)
Q_torch = torch.randn(batch, num_heads, seq_len, head_dim)
K_torch = torch.randn(batch, num_heads, seq_len, head_dim)
V_torch = torch.randn(batch, num_heads, seq_len, head_dim)

Q_tt = ttnn.from_torch(Q_torch, dtype=ttnn.bfloat16, layout=ttnn.TILE_LAYOUT, device=device)
K_tt = ttnn.from_torch(K_torch, dtype=ttnn.bfloat16, layout=ttnn.TILE_LAYOUT, device=device)
V_tt = ttnn.from_torch(V_torch, dtype=ttnn.bfloat16, layout=ttnn.TILE_LAYOUT, device=device)

causal_mask = torch.triu(torch.ones(seq_len, seq_len) * float('-inf'), diagonal=1).unsqueeze(0).unsqueeze(0)
causal_mask_tt = ttnn.from_torch(causal_mask, dtype=ttnn.bfloat16, layout=ttnn.TILE_LAYOUT, device=device)

print("\n=== Accuracy Test ===")
output_composite = composite_sdpa(Q_tt, K_tt, V_tt, causal_mask_tt)
output_composite_torch = ttnn.to_torch(output_composite)[:, :, :seq_len, :head_dim]

output_optimized = ttnn.transformer.scaled_dot_product_attention(Q_tt, K_tt, V_tt, is_causal=True)
output_optimized_torch = ttnn.to_torch(output_optimized)[:, :, :seq_len, :head_dim]

output_torch = torch.nn.functional.scaled_dot_product_attention(Q_torch, K_torch, V_torch, is_causal=True)

pcc_composite = torch.corrcoef(torch.stack([output_composite_torch.flatten(), output_torch.flatten()]))[0, 1].item()
pcc_optimized = torch.corrcoef(torch.stack([output_optimized_torch.flatten(), output_torch.flatten()]))[0, 1].item()
rmse_composite = torch.sqrt(((output_composite_torch - output_torch) ** 2).mean()).item()
rmse_optimized = torch.sqrt(((output_optimized_torch - output_torch) ** 2).mean()).item()

print(f"Composite vs PyTorch:  PCC={pcc_composite:.6f}, RMSE={rmse_composite:.6f}")
print(f"Optimized vs PyTorch:  PCC={pcc_optimized:.6f}, RMSE={rmse_optimized:.6f}")
print("\n=== Speed Test ===")
print("Warming up (compiling kernels)...")
for _ in range(warmup_iterations):
    out = composite_sdpa(Q_tt, K_tt, V_tt, causal_mask_tt)
    out = ttnn.transformer.scaled_dot_product_attention(Q_tt, K_tt, V_tt, is_causal=True)

start = time.perf_counter()
for _ in range(num_iterations):
    output = composite_sdpa(Q_tt, K_tt, V_tt, causal_mask_tt)
ttnn.synchronize_device(device)
composite_time = (time.perf_counter() - start) / num_iterations * 1000

start = time.perf_counter()
for _ in range(num_iterations):
    output = ttnn.transformer.scaled_dot_product_attention(Q_tt, K_tt, V_tt, is_causal=True)
ttnn.synchronize_device(device)
optimized_time = (time.perf_counter() - start) / num_iterations * 1000

speedup = composite_time / optimized_time

print(f"Composite SDPA: {composite_time:.3f} ms")
print(f"Optimized SDPA: {optimized_time:.3f} ms")
print(f"Speedup:        {speedup:.2f}x")

## Math fidelity

TT-NN allows you to have more control over precision of your computations for some operations. 

For example matrix engine has Math Fidelity. It allows you lower precision, but improve performance.

You can find more details [here](https://github.com/tenstorrent/tt-metal/blob/main/tech_reports/matrix_engine/matrix_engine.md) and [here](https://docs.tenstorrent.com/pybuda/latest/dataformats.html)

In [ ]:
import torch
import time

M, K, N = 2048, 2048, 2048
print(f"> Matrix dimensions: {M}x{K} @ {K}x{N}")

torch.manual_seed(42)
a = torch.randn((M, K), dtype=torch.bfloat16)
b = torch.randn((K, N), dtype=torch.bfloat16)
reference = torch.matmul(a.float(), b.float()) 

tt_a = ttnn.from_torch(a, dtype=ttnn.bfloat16, layout=ttnn.TILE_LAYOUT, device=device)
tt_b = ttnn.from_torch(b, dtype=ttnn.bfloat16, layout=ttnn.TILE_LAYOUT, device=device)

print("\n" + "-" * 80)
print(f"{'Fidelity':<10} {'Time (ms)':<12} {'Mean Error':<12}")
print("-" * 80)

# Test different fidelities
fidelities = [
    (ttnn.MathFidelity.LoFi, "LoFi"),
    (ttnn.MathFidelity.HiFi2, "HiFi2"),
    (ttnn.MathFidelity.HiFi3, "HiFi3"),
    (ttnn.MathFidelity.HiFi4, "HiFi4"),
]

for fidelity, name in fidelities:
    # Configure compute kernel
    # Note: Enable FP32 accumulation for HiFi2/HiFi4 to see accuracy benefits
    # With BF16 accumulation and large values, LSB corrections can introduce noise
    use_fp32_acc = (fidelity != ttnn.MathFidelity.LoFi)
    
    config = ttnn.WormholeComputeKernelConfig(
        math_fidelity=fidelity,
        math_approx_mode=False,
        fp32_dest_acc_en=use_fp32_acc,  # FP32 for HiFi2/HiFi4
        packer_l1_acc=use_fp32_acc,     # L1 accumulation for better precision
    )
    
    # Warm-up
    _ = ttnn.matmul(tt_a, tt_b, compute_kernel_config=config)
    
    # Time the operation
    start = time.time()
    for _ in range(50):
        result_tt = ttnn.matmul(tt_a, tt_b, compute_kernel_config=config)
    ttnn.synchronize_device(device)
    elapsed = (time.time() - start) / 50 * 1000  # Convert to ms
    
    # Get result
    result = ttnn.to_torch(result_tt).float()
    
    # Compute errors and PCC
    error = torch.abs(reference - result)
    mean_err = error.mean().item()
    
    print(f"{name:<10} {elapsed:>10.4f}   {mean_err:>10.8f}")

## Metal trace

Allows to trace all the operations and then repeat them again: 

```python
tid = ttnn.begin_trace_capture(device, cq_id=0)  
output = run_model(input)  
ttnn.end_trace_capture(device, tid, cq_id=0)  
ttnn.execute_trace(device, tid, cq_id=0)

```

## Multi-device

You can shard tensors over multiple devices and perform CCL (collective communication) operations

```python
# Open a 1x2 MeshDevice  
mesh_device = ttnn.open_mesh_device(ttnn.MeshShape(1, 2))  
  
# Create a torch tensor  
torch_tensor = torch.zeros(1, 1, 32, 64)  
torch_tensor[..., 0:32] = 1.0  
torch_tensor[..., 32:64] = 2.0  
  
# Shard the tensor across devices along dimension 3  
mesh_tensor = ttnn.from_torch(  
    torch_tensor,  
    layout=ttnn.TILE_LAYOUT,  
    device=mesh_device,  
    mesh_mapper=ttnn.ShardTensorToMesh(mesh_device, dim=3),  
)

output_tensor = ttnn.all_gather(mesh_tensor, dim=3, num_links=1)
```